In [ ]:
import torch
torch.set_default_dtype(torch.float64)
import tdg

import tdg.HMC as HMC
from tqdm.notebook import tqdm

import tdg.plot as visualize
import matplotlib.pyplot as plt

import h5py as h5

Let us try to compare against some data in [2212.05177](https://arxiv.org/abs/2212.05177).

# Target

In [ ]:
# See eq. (7.6)

def alpha3(alpha):
    
    log4 = torch.log(torch.tensor(4.))
    return 0.25 * alpha**2 * (
                    1
                    + (1.5 - log4) * alpha
                    + 3*(0.16079 - (log4-1))*alpha**2
                    # + O(alpha^3)
    )

# Extracted from their Figure 12.

# alpha3 = torch.tensor([
#     [0.598902, 0.074818],[0.511862, 0.057783],[0.444455, 0.045412],[0.361013, 0.031059],
#     [0.305007, 0.022658],[0.250488, 0.015285],[0.177552, 0.007921],[0.125091, 0.003866],
#     [0.056770, 0.000702],[0.000282, -0.000084],[-0.061888, 0.000928],[-0.112767, 0.003178],
#     [-0.183703, 0.008094],[-0.243079, 0.013748],[-0.281439, 0.018160],[-0.315505, 0.022290],
#     [-0.355172, 0.027626],[-0.401856, 0.034315],[-0.435591, 0.038935],[-0.482788, 0.045873],
#     [-0.526242, 0.052000],[-0.568489, 0.057922],[-0.614529, 0.063739],[-0.651008, 0.067718],])



MCDataPositive = torch.tensor([[0.561347, 0.070021],[0.473085, 0.051458],[0.382930, 0.034817],
                               [0.291280, 0.020615],[0.195786, 0.009559],[0.098755, 0.002458],])

MCDataNegative = torch.tensor([[-0.622725, 0.067143],[-0.490006, 0.047254],[-0.395163, 0.032882],
                               [-0.309663, 0.019606],[-0.255783, 0.014222],[-0.234334, 0.012622],])

def comparison_plot(ax):
    x = torch.linspace(-0.65,+0.65,1000)
    ax.plot(x, alpha3(x), color='black')
    ax.fill_between(x, alpha3(0.95*x), alpha3(1.05*x), color='gray', alpha=0.2)
    ax.plot(MCDataNegative[:,0], MCDataNegative[:,1], color='gray', marker='o', linestyle='none')
    ax.plot(MCDataPositive[:,0], MCDataPositive[:,1], color='blue', marker='v', linestyle='none')
    ax.set_xlim([-0.65,+0.65])
    ax.set_ylim([0,0.075])    

fig, ax = plt.subplots(1,1, figsize=(10,6))
comparison_plot(ax)

# Tuning

First, to establish their notation:

$$\begin{align}
    \alpha(x) &= -\left(\log x a\right)^{-1} 
    &
    \alpha    &= \alpha(k_F) \text{ if no argument is provided.}
\end{align}$$

We should aim for $\alpha>0$ so that we do not need to compute the gap $M \Delta^2/4\pi$ (5.1).  Their results are good up to $\alpha \lesssim 0.6$ or so.

What does that correspond to in terms of the number density
$$ \rho = N/L^2 = \frac{g}{4\pi} k_F^2 ?$$

Massaging the definition of $\alpha$ one finds
$$\begin{align}
    e^{-1/\alpha} = k_F a = \sqrt{\frac{4\pi N}{gL^2}} a = \sqrt{\frac{N}{g\pi}} \frac{2\pi a}{L}
\end{align}$$

Since for our system $g=2$,
$$\begin{align}
    e^{-1/\alpha} = \sqrt{\frac{N}{2\pi}} \frac{2\pi a}{L} = \sqrt{\frac{N}{2\pi}} \tilde{a}
\end{align}$$

Suppose we want to stay in the dilute regime $\text{sparsity} = N/\Lambda\lesssim 0.1 $ to keep spatial discretization errors at bay.  Then we should solve
$$\begin{align}
    e^{-1/\alpha} = \sqrt{\frac{\text{sparsity} \Lambda}{2\pi}} \tilde{a}
\end{align}$$
for $\tilde{a}$ to find
$$\begin{align}
    \tilde{a} = \frac{e^{-1/\alpha}}{\sqrt{\frac{\Lambda \text{sparsity}}{2\pi}}}
\end{align}$$

In [ ]:
def target_ere(alpha, lattice, sparsity):
    
    atilde = torch.exp(-1/alpha) / torch.sqrt( lattice.sites * sparsity / (2*torch.pi))
    
    return tdg.EffectiveRangeExpansion(torch.tensor((atilde,)))

They show the contact density [their (7.5)] for a range of α in Fig. 12.  Something to note is that the energy density in their definition [their (5.1)] requires subtracting off the gap.

We can access the attractive channel with α < 0.  

In [ ]:
alpha_target = torch.tensor(-0.3)
Lattice = tdg.Lattice(7)
sparsity = torch.tensor(0.1) # so we should tune µ until we find about 5 particles
Z = tdg.Luescher.Zeta2D()

We can tune to the continuum limit.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

# First draw the zeta function.
exact = torch.linspace(-5.001, 30.001, 1000)
Z.plot(ax, exact, color='gray')

# and the analytic piece of the ERE
ere = target_ere(alpha_target, Lattice, sparsity)
analytic = ere.analytic(exact)
ax.plot(exact, analytic.clone().detach(), color='black', linestyle='dashed')

def plot(ax, nx, alpha, starting_lattice, starting_sparsity):
    Lattice = tdg.Lattice(nx)
    # Take the continuum limit holding the number of particles fixed
    # and increasing the number of sites.
    sparsity = starting_sparsity*(starting_lattice.nx/nx)**2
    
    ere = target_ere(alpha, Lattice, sparsity)
    tuning = tdg.AnalyticTuning(ere, Lattice)
    A1 = tdg.ReducedTwoBodyA1Hamiltonian(Lattice, [tdg.LegoSphere(r) for r in tuning.radii])
    E  = A1.eigenenergies(tuning.C)
    x  = E * Lattice.sites / (2*torch.pi)**2
    z  = Z(x) / torch.pi**2
    ax.plot(
        x.clone().detach().numpy(),
        z.clone().detach().numpy(),
        linestyle='none', marker='o',
        label=f'nx={Lattice.nx}'
        )

for nx in range(7,25,2):
    plot(ax, nx, alpha_target, Lattice, sparsity)

ax.legend()
ax.set_xlim((min(exact), max(exact)))
ax.set_ylim((-0,5))

# Many-Body

Now that we know the Hamiltonian would converge to the right thing, let us prepare a many-body calculation!

In [ ]:
tuning = tdg.AnalyticTuning(ere, Lattice)
nt=32
beta = 3
mu = 0. # Iterated over a few choices until N came out about right.  Is it a coincidence that it's 0?

In [ ]:
S = tuning.Action(nt, beta, torch.tensor(mu))
print(S)

In [ ]:
H = HMC.Hamiltonian(S)
integrator = HMC.Omelyan(H, 30, 1)
hmc = HMC.MarkovChain(H, integrator)

In [ ]:
configurations = 1000 # takes about 20 minutes on my laptop

# Read from a stored ensemble if possible.
try:
    with h5.File('./many-body-contact-comparison.h5', 'r') as f:
        ensemble = tdg.ensemble.GrandCanonical.from_h5(f['/ensemble'])
        if len(ensemble.configurations) < configurations:
            raise
        else:
            ensemble.configurations = ensemble.configurations[-configurations:]
except:
    ensemble = tdg.ensemble.GrandCanonical(S).generate(configurations, hmc, start='hot', progress=tqdm)
    with h5.File('./many-body-contact-comparison.h5', 'w') as f:
        ensemble.to_h5(f.create_group('/ensemble'))

In [ ]:
viz = visualize.History(2)
viz.plot(ensemble.N('fermionic').real, 0, label=f'N = {ensemble.N("fermionic").mean().real:.4f}')
viz.plot(ensemble.S.real, 1)
viz.ax[0,0].legend()

# Analysis

In [ ]:
binned = ensemble.cut(100).binned(16)
bootstrapped = binned.bootstrapped()

In [ ]:
viz = visualize.History(2)
viz.plot(ensemble.N('fermionic').real, 0, label=f'N = {ensemble.N("fermionic").mean().real:.4f}')
viz.plot(ensemble.S.real, 1, label=f'S = {binned.S.real.mean():.4f}')

viz.plot(binned.N('fermionic').real, 0, x=binned.index, label=f'binned = {binned.N("fermionic").mean().real:.4f}')
viz.plot(binned.S.real, 1, x=binned.index, label=f'binned = {binned.S.real.mean():.4f}')

viz.ax[0,0].legend()
viz.ax[1,0].legend()

Now we can evaluate $k_F a = \sqrt{N/\pi g} \tilde{a}$ and check where we got α close to our target value, given the β and μ we picked.

In [ ]:
bootstrapped.kFa = torch.sqrt(bootstrapped.N('fermionic') / (2*torch.pi)) * ere.a
bootstrapped.alpha = -1./torch.log(bootstrapped.kFa)

In [ ]:
def err(bootstrapped_observable, n = None):
    mean = bootstrapped_observable.real.mean()
    std  = bootstrapped_observable.real.std()
    
    if std < 1:
        precision = int(-torch.log10(std)) + 1
    else:
        precision = 1
        
    return f'{mean:.{precision}f} ± {std:.{precision}f}'

In [ ]:
print(f'kFa {err(bootstrapped.kFa)}')
print(f'α {alpha_target}             target')
print(f'α {err(bootstrapped.alpha)} measured')

Great!  A close value!  Let's look at the contact!

In [ ]:
viz = visualize.History()
viz.plot(ensemble.contact('fermionic').real, 0, label=f'contact ∆x^2 = {ensemble.contact("fermionic").mean().real:.4f}')
viz.plot(binned.contact('fermionic').real, 0, x=binned.index, label=f'binned = {binned.contact("fermionic").mean().real:.4f}')
viz.ax[0,0].legend()

Check the error has plateaued:

In [ ]:
cut = ensemble.cut(100) # Share the measurements among every binning.

viz = visualize.ScatterTriangle(3, labels=('(2πN)^2', 'Λ C ∆x^2', 'C/kF^4'))

for width in [2, 4, 8, 16, 24, 32]:
    binned = cut.binned(width)
    
    viz.plot((
        (2*torch.pi*binned.N('fermionic')).real**2, 
        ensemble.Action.Spacetime.Lattice.sites*binned.contact('fermionic').real,
        (ensemble.Action.Spacetime.Lattice.sites*binned.contact('fermionic').real
         / (2*torch.pi*binned.N('fermionic')).real**2
        )
    ))
    
    plateau = binned.bootstrapped()
    
    # See below for why this is a good observable to compute.
    plateau.C_by_kF4 = (
        ensemble.Action.Spacetime.Lattice.sites
        * plateau.contact('fermionic')
        / (2*torch.pi * plateau.N('fermionic'))**2
    )
    print(f'{width =:3}    bins = {binned.bins:3}    C/kF^4 = {err(plateau.C_by_kF4)}')
    
viz.grid[1,0].plot(
    (400, 1200), (0.0228*400, 0.0228*1200),
    color='gray', alpha=0.5)

low = plateau.C_by_kF4.real.mean()-plateau.C_by_kF4.real.std()
high= plateau.C_by_kF4.real.mean()+plateau.C_by_kF4.real.std()

viz.grid[2,0].axhspan(
    low, high,
    color='gray', alpha=0.5, zorder = -1)

Looks like a reasonable uncertainty estimate!

# Comparison

Beane et al. report $C/k_F^4$ in their Figure 12.

Their $C$ is the contact *density*.  

To make a fair comparison we should compute $\Lambda (C \Delta x^2) / (2 \pi N)^2$.

In [ ]:
bootstrapped.C_by_kF4 = (
    ensemble.Action.Spacetime.Lattice.sites
    * bootstrapped.contact('fermionic')
    / (2*torch.pi * bootstrapped.N('fermionic'))**2
)

In [ ]:
print(f"C/kF^4 = {err(bootstrapped.C_by_kF4)}")

Seems to be compatible with the curve in Figure 12 of [2212.05177](https://arxiv.org/abs/2212.05177).

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6))

comparison_plot(ax)

ax.errorbar(
    (bootstrapped.alpha.real.mean().detach().numpy(),),
    (bootstrapped.C_by_kF4.real.mean().detach().numpy(),),
    xerr=(bootstrapped.alpha.real.std().detach().numpy(),),
    yerr=(bootstrapped.C_by_kF4.real.std().detach().numpy(),),
    color='red', marker=None
)

So we're within the ±5% error band of the EFT prediction, without spatial continuum, infinite volume, or cold limits.  That's extremely promising!

In their notation, however, the contact density is the derivative of $\mathcal{E}_{FL}$ rather than $\mathcal{E}$ (a Hamiltonian eigenvalue).  The difference is, when $\alpha < 0$,
$$
    \mathcal{E}_{FL} = \mathcal{E} + \frac{M \Delta^2}{4\pi}
$$

Is this difference detectable at our level of uncertainty?

The difference between our two calculations of $C/k_F^4$ is (up to a sign)
$$
    \text{difference}
    = \frac{1}{k_F^4} 2\pi M \frac{d}{d\log a} \left( \frac{M \Delta^2}{4\pi} \right)
    = \frac{1}{2 k_F^4} \frac{d}{d\log a}\left( M^2 \Delta^2 \right)
$$
Now, since
$$
\begin{align}
    \Delta^2 &= 2 E_F |E_B|
    &
    E_F &= \frac{k_F^2}{2M}
    &
    E_B &= - \frac{1}{Ma^2}
\end{align}
$$
from equations (4.17), (4.3), and (3.20) respectively, we have
$$
    \text{difference}
    = \frac{1}{2 k_F^4} \frac{d}{d\log a}\left( 2 M^2 E_F |E_B| \right)
    = \frac{1}{2 k_F^4} \frac{d}{d\log a}\left( 2 M^2 \frac{k_F^2}{2M} \frac{1}{Ma^2} \right)
$$
and simplifying,
$$
    \text{difference}
    = \frac{1}{2 k_F^4} \frac{d}{d\log a} \left( \frac{k_F^2}{a^2} \right)
    = \frac{1}{(k_F a)^2}\left(\frac{a k_F'}{k_F}-1\right)
$$

If we assume $a k_F'/k_F = d\log k_F / d\log a$ is small the difference is of the size $(k_F a)^{-2}$

In [ ]:
print(f'C/kF^4    {err(bootstrapped.C_by_kF4)}')
print(f'1/(kFa)^2 {err(1/(bootstrapped.kFa)**2)} ~ the difference')

So the discrepancy is 3x our uncertainty!